# Calculation of Metadata Amplification

In [2]:
import pandas as pd
import numpy as np

files = ["./performance-comparison-table-NOVA", "./performance-comparison-table-PMFS"] 
workloads = ["write", "randwrite", "fileserver.f", "varmail.f", "webserver.f", "webproxy.f"]

real_meta_IOs_NOVA = []
real_meta_IOs_PMFS = []
software_meta_IO_percentages_NOVA = []
software_meta_IO_percentages_PMFS = []
meta_times_NOVA = []
meta_times_PMFS = []
meta_time_percentages_NOVA = []
meta_time_percentages_PMFS = []

for fid, file in enumerate(files):
    with open(file, "r") as f:
        df = pd.read_csv(f, delim_whitespace=True, engine='python')
    
    breakdown_times = df.columns[14:]
    breakdown_times = breakdown_times.insert(0, "data_read_time(ns)")
    breakdown_times = breakdown_times.insert(1, "data_write_time(ns)")
    
    print("=====================================")
    print(file)
    for workload_idx, workload in enumerate(workloads):
        total_time = 0
        meta_time = 0
        data_time = 0
        print(breakdown_times)
        for idx, breakdown_time in enumerate(breakdown_times):
            total_time += df[df['workloads'] == workload][breakdown_time].iloc[0]
            if idx == 0 or idx == 1:
                data_time += df[df['workloads'] == workload][breakdown_time].iloc[0]
            else:
                meta_time += df[df['workloads'] == workload][breakdown_time].iloc[0]
        # data_time = df[df['workloads'] == workload]["data_time(ns)"].iloc[0]
        # meta_time = df[df['workloads'] == workload]["meta_time(ns)"].iloc[0]
        # total_time = data_time + meta_time

        print("Total IO time of", workload, ": ", total_time / 1000000000)
        print("Meta IO time of", workload, ": ", meta_time / 1000000000)
        print("Data IO time of", workload, ": ", data_time / 1000000000)
        print("Meta IO time percentage of", workload, ": ", meta_time * 100 / total_time, "%")
        print("Data IO time percentage of", workload, ": ", data_time * 100 / total_time, "%")

        for breakdown_time in breakdown_times:
            print(breakdown_time + ": " + str(df[df['workloads'] == workload][breakdown_time].iloc[0] / 1000000000) + "s")
            print(breakdown_time + ": " + str(df[df['workloads'] == workload][breakdown_time].iloc[0] * 100 / total_time) + "%")

        data_write = df[df['workloads'] == workload]["data_write(bytes)"].iloc[0]
        data_read = df[df['workloads'] == workload]["data_read(bytes)"].iloc[0]
        media_write = df[df['workloads'] == workload]["media_write(byte)"].iloc[0]
        media_read = df[df['workloads'] == workload]["media_read(byte)"].iloc[0]
        meta_write = df[df['workloads'] == workload]["meta_write(bytes)"].iloc[0]
        meta_read = df[df['workloads'] == workload]["meta_read(bytes)"].iloc[0]

        software_meta_IO = meta_write + meta_read
        real_meta_IO = 0 if media_write + media_read - media_write - data_write - data_read < 0 else media_write + media_read - media_write - data_write - data_read 
        software_total_IO = software_meta_IO + data_write + data_read

        if fid == 0:
            real_meta_IOs_NOVA.append(real_meta_IO)
        else:
            real_meta_IOs_PMFS.append(real_meta_IO)

        print("workload: ", workload)
        print("software_meta_IO: ", software_meta_IO / 1024 / 1024 / 1024)
        print("real_meta_IO: ", real_meta_IO / 1024 / 1024 / 1024)
        print("software_meta_IO percentage: ", software_meta_IO * 100 / software_total_IO, "%")

        software_meta_IO_percentages_NOVA.append(software_meta_IO * 100 / software_total_IO) if fid == 0 else software_meta_IO_percentages_PMFS.append(software_meta_IO * 100 / software_total_IO)
        meta_times_NOVA.append(meta_time / 1000000000) if fid == 0 else meta_times_PMFS.append(meta_time / 1000000000)
        meta_time_percentages_NOVA.append(meta_time * 100 / total_time) if fid == 0 else meta_time_percentages_PMFS.append(meta_time * 100 / total_time)


    # for workload_idx, workload in enumerate(workloads):

def format_nice(array):
    return [round(x, 2) for x in array]

print("software_meta_IO_percentages_NOVA: ", format_nice(software_meta_IO_percentages_NOVA))
print("meta_time_percentages_NOVA: ", format_nice(meta_time_percentages_NOVA))
print("software_meta_IO_percentages_PMFS: ", format_nice(software_meta_IO_percentages_PMFS))
print("meta_time_percentages_PMFS: ", format_nice(meta_time_percentages_PMFS))
# print("meta_times_NOVA: ", meta_times_NOVA)
# print("meta_times_PMFS: ", meta_times_PMFS)

./performance-comparison-table-NOVA
Index(['data_read_time(ns)', 'data_write_time(ns)', 'update_entry_time(ns)',
       'update_inode_time(ns)', 'journal_time(ns)',
       'update_page_tail_time(ns)'],
      dtype='object')
Total IO time of write :  42.375228559
Meta IO time of write :  31.990792054
Data IO time of write :  10.384436505
Meta IO time percentage of write :  75.49408732854027 %
Data IO time percentage of write :  24.505912671459722 %
data_read_time(ns): 0.0s
data_read_time(ns): 0.0%
data_write_time(ns): 10.384436505s
data_write_time(ns): 24.505912671459722%
update_entry_time(ns): 11.944316366s
update_entry_time(ns): 28.187025231898527%
update_inode_time(ns): 3.958139137s
update_inode_time(ns): 9.340690945156773%
journal_time(ns): 1.306e-06s
journal_time(ns): 3.081989276309451e-06%
update_page_tail_time(ns): 16.088335245s
update_page_tail_time(ns): 37.9663680694957%
workload:  write
software_meta_IO:  4.1812749253585935
real_meta_IO:  11.62866735458374
software_meta_IO per

# KILLER

In [6]:
import pandas as pd
import numpy as np

files = ["./performance-comparison-table-KILLER"] 
workloads = ["write", "randwrite", "fileserver.f", "varmail.f", "webserver.f", "webproxy.f"]
real_meta_IOs_of_KILLER = []
package_persist_time_percentages = []

for fid, file in enumerate(files):
    with open(file, "r") as f:
        df = pd.read_csv(f, delim_whitespace=True, engine='python')

    for workload_idx, workload in enumerate(workloads):
        data_write = df[df['workloads'] == workload]["data_write(bytes)"].iloc[0]
        data_read = df[df['workloads'] == workload]["data_read(bytes)"].iloc[0]
        media_write = df[df['workloads'] == workload]["media_write(byte)"].iloc[0]
        media_read = df[df['workloads'] == workload]["media_read(byte)"].iloc[0]
        meta_write = df[df['workloads'] == workload]["meta_write(bytes)"].iloc[0]
        meta_read = df[df['workloads'] == workload]["meta_read(bytes)"].iloc[0]

        data_time = df[df['workloads'] == workload]["data_time(ns)"].iloc[0]
        meta_time = df[df['workloads'] == workload]["meta_time(ns)"].iloc[0]
        total_time = data_time + meta_time

        meta_time = df[df['workloads'] == workload]["meta_time(ns)"].iloc[0]
        update_bm_time = df[df['workloads'] == workload]["update_bm_time(ns)"].iloc[0]
        update_package_time = df[df['workloads'] == workload]["update_package_time(ns)"].iloc[0]

        software_meta_IO = meta_write + meta_read
        real_meta_IO = 0 if media_write + media_read - media_write - data_write - data_read < 0 else media_write + media_read - media_write - data_write - data_read 
        software_total_IO = software_meta_IO + data_write + data_read
        
        real_meta_IOs_of_KILLER.append(real_meta_IO)

        print("workload: ", workload)
        print("software_meta_IO: ", software_meta_IO / 1024 / 1024 / 1024)
        print("real_meta_IO: ", real_meta_IO / 1024 / 1024 / 1024)
        print("software_meta_IO percentage: ", software_meta_IO * 100 / software_total_IO, "%")
        print("Total IO time (seconds) of", workload, ": ", total_time / 1000000000)
        print("meta percentage: ", meta_time * 100 / total_time, "%")
        print("meta time: ", meta_time / 1000000000, "s")
        print("bm percentage: ", update_bm_time * 100 / total_time, "%")
        print("bm time: ", update_bm_time / 1000000000, "s")
        print("package percentage: ", update_package_time * 100 / total_time, "%")
        print("package time: ", update_package_time / 1000000000, "s")
        print("")
        package_persist_time_percentages.append(update_package_time * 100 / total_time)
        # software_meta_IO = meta_write + meta_read
        # real_meta_IO = 0 if media_write + media_read - media_write - data_write - data_read < 0 else media_write + media_read - media_write - data_write - data_read 

        # print("workload: ", workload)
        # print("software_meta_IO: ", software_meta_IO / 1024 / 1024 / 1024)
        # print("real_meta_IO: ", real_meta_IO / 1024 / 1024 / 1024)
print("real_meta_IOs_of_KILLER: ", format_nice(real_meta_IOs_of_KILLER))
print("package_persist_time_percentages: ", format_nice(package_persist_time_percentages))

workload:  write
software_meta_IO:  0.06335536669939756
real_meta_IO:  0.7195720672607422
software_meta_IO percentage:  0.19759431280606915 %
Total IO time (seconds) of write :  10.29471369
meta percentage:  32.86545324020565 %
meta time:  3.383404314 s
bm percentage:  0.0021798080719620284 %
bm time:  0.000224405 s
package percentage:  32.86327343213369 %
package time:  3.383179909 s

workload:  randwrite
software_meta_IO:  0.5001223925501108
real_meta_IO:  2.960953712463379
software_meta_IO percentage:  1.5388323357968403 %
Total IO time (seconds) of randwrite :  21.448647251
meta percentage:  46.913967632764624 %
meta time:  10.062411429 s
bm percentage:  0.25836513767746516 %
bm time:  0.055415827 s
package percentage:  46.65560249508716 %
package time:  10.006995602 s

workload:  fileserver.f
software_meta_IO:  0.2417338714003563
real_meta_IO:  3.5799068277701735
software_meta_IO percentage:  0.2715133740555585 %
Total IO time (seconds) of fileserver.f :  39.763399119
meta percent

# Reduced Metadata I/Os 

In [4]:
min_amps = []
max_amps = []
for idx, meta_IO in enumerate(real_meta_IOs_of_KILLER):
    if meta_IO == 0:
        continue
    amp_NOVA = real_meta_IOs_NOVA[idx] / meta_IO 
    amp_PMFS = real_meta_IOs_PMFS[idx] / meta_IO
    min_amp = min(amp_NOVA, amp_PMFS)
    min_amps.append(min_amp)
    max_amp = max(amp_NOVA, amp_PMFS)
    max_amps.append(max_amp)
    
print("min_amps: ", min_amps)
print("max_amps: ", max_amps)

min_amps:  [11.169330148994202, 4.409876770855309, 1.9999575680472934, 1.2297011752517892, 1.113008177335789]
max_amps:  [16.160531910099852, 4.524832154239343, 2.634641822682739, 1.9096854456278018, 1.359348929958735]
